# IBM Db2 

In this notebook I will be using a Db2 database to create a dataset of football players with certain characteristics. 



## Cloud Databases

A Cloud database is a database service built and accessed through a Cloud platform. It has many of the same functions as traditional databases with the added flexibility of Cloud computing. 
Some advantages of using Cloud databases:
* Ease of use: Users can accesss these databases from virtually everywhere using a vendors API or web interface, or your own applications. 
* Scalability: They can expand and shrink their storage and compute capacities during runtime to accomodate changing needs and usage demands, so you only pay for what you need. 
* Disaster Recovery: Data is kept secure through backups on remote servers on cloud in geographically distributed regions. 



In [1]:
!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 kB 11.9 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - \ done
  Installing backend dependencies ... - \ done
  Preparing metadata (pyproject.toml) ... - done
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 21.3 MB/s eta 0:00:00
  Created wheel for ibm_db: filename=ibm_db-3.1.0-cp310-cp310-linux_x86_64.whl size=41726101 sha256=3d177f865ca4f3bdfc1fc9501ea79e387877d7f3226b9eb436f095e1c9c22e58
  Stored in directory: /root/.cache/pip/wheels/50/d7/3d/cf01ca490baa2f299aca0c791a2a5708d9b9676608374e2f31
  Created wheel for ibm_db_sa: filename=ibm_db_sa-0.3.3-py3-none-any.whl size=27419 sha256=9ef6c265e91a37c7797468007a22fb5c3b71f07f797fb3a147926e98a0a3bb9a
  Stored in directory: /root/.cache/pip/wheels/63/28/d7/aea2c85d7ff99e1e8d

In [2]:
!pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24

Found existing installation: SQLAlchemy 2.0.30
Uninstalling SQLAlchemy-2.0.30:
  Successfully uninstalled SQLAlchemy-2.0.30
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.24-cp310-cp310-linux_x86_64.whl size=1220150 sha256=e0d4a54a2b953b01679a1b1639a2073e748e4a7ed772884c407b3f3d41f7fbdf
  Stored in directory: /root/.cache/pip/wheels/27/51/b3/3481e88d5a5ba95dd4aafedc9316774d941c4ba61cfb93add8
Successfully built sqlalchemy
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.24 which is incompatible.


In [3]:
!pip install ipython-sql

  Using cached SQLAlchemy-2.0.30-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
Using cached SQLAlchemy-2.0.30-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.24
    Uninstalling SQLAlchemy-1.3.24:
      Successfully uninstalled SQLAlchemy-1.3.24


In [4]:
import ibm_db

In [5]:
%load_ext sql 

### Database Connection Credentials 

In [6]:
dsn_hostname = "2d46b6b4-cbf6-40eb-bbce-6251e6ba0300.bs2io90l08kqb1od8lcg.databases.appdomain.cloud" # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "vpc61689"        # e.g. "abc12345"
dsn_pwd = "jboiCdMPKZLcYWAC"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "32328"                # e.g. "32733" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"

### Connect DB2 Database Connection

In [7]:
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=2d46b6b4-cbf6-40eb-bbce-6251e6ba0300.bs2io90l08kqb1od8lcg.databases.appdomain.cloud;PORT=32328;PROTOCOL=TCPIP;UID=vpc61689;PWD=jboiCdMPKZLcYWAC;SECURITY=SSL;


### Establish Connections to the database

In [8]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create database connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  vpc61689 on host:  2d46b6b4-cbf6-40eb-bbce-6251e6ba0300.bs2io90l08kqb1od8lcg.databases.appdomain.cloud


### Metadata for Database server

In [9]:
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.05.0900
DB_NAME:    BLUDB


### Metadata for the Database Client/Driver

In [10]:
client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

DRIVER_NAME:           libdb2.a
DRIVER_VER:            11.05.0900
DATA_SOURCE_NAME:      BLUDB
DRIVER_ODBC_VER:       03.51
ODBC_VER:              03.01.0000
ODBC_SQL_CONFORMANCE:  EXTENDED
APPL_CODEPAGE:         819
CONN_CODEPAGE:         1208


Need to delete previous instances of table

In [11]:
#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt
dropQuery = "drop table FOOTBALLERS"

#Now execute the drop statment
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

### Creating a table in the database

In [12]:
createQuery = "create table FOOTBALLERS(id INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), AGE INT, POSITION CHAR(2), LEAGUE VARCHAR(20), TEAM VARCHAR(20), WAGE INT)"
createStmt = ibm_db.exec_immediate(conn, createQuery)

In [13]:
insertQuery = "insert into FOOTBALLERS values (1, 'Bukayo', 'Saka', 22, 'RW', 'Premier League', 'Arsenal', 295000)"


insertStmt = ibm_db.exec_immediate(conn, insertQuery)

In [14]:
insertQuery2 = "insert into FOOTBALLERS values (2, 'Declan', 'Rice', 25, 'CM', 'Premier League', 'Arsenal', 250000), (3, 'Kylian', 'Mbappe', 25, 'LW','Ligue 1', 'PSG', 630000), (4, 'Jude', 'Bellingham', 20, 'CM', 'La Liga','Real Madrid', 220000), (5, 'Phil', 'Foden', 23, 'CM', 'Premier League', 'Man City', 200000), (6, 'Marcus', 'Rashford', 26, 'LW', 'Premier League', 'Man United', 300000)"

insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)

In [15]:
selectQuery = "select * from FOOTBALLERS"

selectStmt = ibm_db.exec_immediate(conn, selectQuery)

ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Bukayo',
 1: 'Bukayo',
 'LNAME': 'Saka',
 2: 'Saka',
 'AGE': 22,
 3: 22,
 'POSITION': 'RW',
 4: 'RW',
 'LEAGUE': 'Premier League',
 5: 'Premier League',
 'TEAM': 'Arsenal',
 6: 'Arsenal',
 'WAGE': 295000,
 7: 295000}

### Retrieving Data into Pandas

In [16]:
import pandas as pd 
import ibm_db_dbi

In [17]:
# Connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [18]:
 
selectQuery = "select * from FOOTBALLERS"

pdf = pd.read_sql(selectQuery, pconn)

pdf.LNAME[2]

/tmp/ipykernel_18/532998386.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(selectQuery, pconn)


'Mbappe'

In [19]:
pdf

,ID,FNAME,LNAME,AGE,POSITION,LEAGUE,TEAM,WAGE
0,1,Bukayo,Saka,22,RW,Premier League,Arsenal,295000
1,2,Declan,Rice,25,CM,Premier League,Arsenal,250000
2,3,Kylian,Mbappe,25,LW,Ligue 1,PSG,630000
3,4,Jude,Bellingham,20,CM,La Liga,Real Madrid,220000
4,5,Phil,Foden,23,CM,Premier League,Man City,200000
5,6,Marcus,Rashford,26,LW,Premier League,Man United,300000


In [20]:
pdf.shape

(6, 8)

In [21]:
pdf.describe

<bound method NDFrame.describe of    ID   FNAME       LNAME  AGE POSITION          LEAGUE         TEAM    WAGE
0   1  Bukayo        Saka   22       RW  Premier League      Arsenal  295000
1   2  Declan        Rice   25       CM  Premier League      Arsenal  250000
2   3  Kylian      Mbappe   25       LW         Ligue 1          PSG  630000
3   4    Jude  Bellingham   20       CM         La Liga  Real Madrid  220000
4   5    Phil       Foden   23       CM  Premier League     Man City  200000
5   6  Marcus    Rashford   26       LW  Premier League   Man United  300000>

### Always close connection :)

In [22]:
ibm_db.close(conn)

True